In [1]:
import os
import nltk
import base64
from base64 import b64encode, b64decode
import uuid
from IPython.display import display, HTML
from unstructured.partition.pdf import partition_pdf
from operator import itemgetter
from pydantic import Field
from typing import List, TypedDict

from langchain_core.documents import Document
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_classic.retrievers import MultiVectorRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_core.stores import BaseStore
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage
from langchain_core.stores import InMemoryStore
from langchain_openai import ChatOpenAI
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma

In [2]:
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [3]:
file_path = './data/mal_sample.pdf'

raw_pdf_elements = partition_pdf(
    filename=file_path,
    extract_images_in_pdf=True,
    infer_table_structure=True,
    chunking_strategy='by_title',
    extract_image_block_output_dir='./data/mal_sample_picture_2'
)

The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


In [9]:
tables = []
texts = []

for element in raw_pdf_elements:
    if 'unstructured.documents.elements.Table' in str(type(element)):
        tables.append(str(element))
    elif 'unstructured.documents.elements.CompositeElement' in str(type(element)):
        texts.append(str(element))

In [10]:
print(tables[0])
print('--'*20)
print(texts[0])

202314 (1. 1.~12. 31.) A CHHI(4278) 20234 (1. 1.~10. 21.) (1. 202414 1.~10. 19.) Ail 673 (100) 663 (100.0) 630 (100.0) 5.0% ae tt 569 (84.5) 560 (84.5) 526 (83.5) 16.1% OfAt 104 (15.5) 108 (15.5) 104 (16.5) 1.0% we 0-94] 5 (0.7) 5 (0.8) 2( 0.3) 460.0% 10-194 31 ( 4.6) 30 ( 4.5) 20 ( 3.2) A33.3% 20-29M 201 (29.9) 200 (30.2) 209 (33.2) 4.5% 30-39M| 111 (16.5) 110 (16.6) 90 (14.3) A18.2% 40-494 107 (15.9) 104 (15.7) 96 (15.2) AT.7%
----------------------------------------
42주차 (10.13.~10.19.)

| Suyztay | aygises zt | Bail

주차

국내발생 해외유입

전체


In [11]:
prompt_text = '''
당신은 표와 텍스트를 요약하여 검색할 수 있도록 돕는 역할을 맡은 어시스턴트입니다.
이 요약은 임베딩되어 원본 텍스트나 표 요소를 검색하는데 사용될 것입니다.
표 또는 텍스트에 대한 빠짐 없는 요약을 제공하여 검색에 최적화된 형태로 만들어 주세요.
표 또는 텍스트: {element}'''

prompt = ChatPromptTemplate.from_template(prompt_text)
model = ChatOpenAI(model='gpt-5-mini', temperature=0)
summarize_chain = (
    {'element': lambda x:x}
    | prompt
    | model
    | StrOutputParser()
)

In [12]:
text_summaries = await summarize_chain.abatch(texts, config={'max_concurrency':5})
tables_summaries = await summarize_chain.abatch(tables, config={'max_concurrency':5})